In [90]:
# prompt: process json data where it is in list. Each list elements are a dictionary with key author, content, url.

def process_json_data(json_data):
  with open('twitter_bookmarks.csv', 'a') as f:
    f.write("counter,author,content,url\n")
    for item in json_data:
      counter = item.get('counter', '')
      author = item.get('author', '')
      content = item.get('content', '')
      url = item.get('url', '')
      f.write(f"{counter},{author},{content},{url}\n")


In [91]:
# prompt: make function like process_json_data so it save into json instead of csv, where each element already cleaned from whitespace

def process_json_data_to_json(json_data):
  cleaned_data = []
  counter = 1
  for item in json_data:
    item["counter"] = str(counter)
    counter+=1
    for key in item:
      item[key] = item[key].strip().replace('\n', ' ').replace('\t', ' ').replace(',', ' ')
    cleaned_data.append(item)

  # save to json
  import json
  with open('twitter_bookmarks_cleaned.json', 'w') as f:
    json.dump(cleaned_data, f, indent=4)


In [92]:
# prompt: open json file named twitter_bookmarks.json

import json

with open('twitter_bookmarks.json', 'r') as f:
  json_data = json.load(f)


In [93]:
process_json_data_to_json(json_data)

In [94]:
with open('twitter_bookmarks_cleaned.json', 'r') as f:
  json_data_cleaned = json.load(f)

In [95]:
process_json_data(json_data_cleaned)

In [114]:
# prompt: open the csv file

import pandas as pd
df = pd.read_csv('twitter_bookmarks.csv')
df.tail(5)



,counter,author,content,url
344,345,Namya @ Supafast @namyakhann · Aug 25 2023,As requested here're 12 gray alternatives for...,https://x.com/namyakhann/status/16950056209766...
345,346,ʞǝǝǝǝʎǝԀ @peyeeeek · Aug 24 2023,Monggo,https://x.com/peyeeeek/status/1694600639664541938
346,347,Gading Nasution ᵍⁿ @gadingnstn · Jul 2 2023,buat deploy BE + Database gratisan. enakan ht...,https://x.com/gadingnstn/status/16754173495273...
347,348,Ahnaf | Sukses Kuliah! @ahnafau · Jun 27 2023,Gimana cara mulai magang saat CV masih kosong?...,https://x.com/ahnafau/status/1673672800131842050
348,349,"Eza ""Disclaimer"" Hazami @ezash · Mar 19 2023",Ada iPhone 5s ga kepake semua fungsi normal d...,https://x.com/ezash/status/1637286495223693312


In [184]:
# prompt: from the csv, process the data as follows:
# content about math append to dataframe math
# content about jkt48, idoling, or japanese append to dataframe idol
# content about coding or tech append to dataframe tech
# other content append to dataframe other

import json
import pandas as pd

# dictionary where the value are dataframes
term_dict = {}
df_dict = {}
name_list = ["math", "idol", "tech", "other"]

term_dict["math"] = ["math", "find ", "geometry", "other", "theorem", "puzzle",
                     "mirangu", "cetin", "solution", "barra", "calculate", "angle",
                     "prove", "question"]
term_dict["idol"] = ["idol", "jkt48", "48", "lana", "erin", "elin", "peyeeeek",
                     "yokanang", "greesel", "kamaniya", "lockscreen", "ofc", "photobook",
                     "cynthia", "wota","jeketi", "pajama", "cepio", "fiony", "aisa",
                     "flora", "maeng", "icel", "kathy", "keti", "gracie", "rohmatkun",
                     "emot", "neru", "andiano", "flyingglunt"]
term_dict["tech"] = ["code","coding", "tech", "program", "web", "memory", "software",
                     "dev", "engineer", "deploy", "backend", "frontend",
                     "html", "javascript", "css", "js"]


In [185]:
for k in name_list:
  df_dict[k] = pd.DataFrame(columns=["counter", "author", "content", "url"])

def classify_df(df, df_dict):
  for index, row in df.iterrows():
    content = str(row['content']).encode('ascii', 'ignore').decode().lower()
    author = str(row['author']).encode('ascii', 'ignore').decode().lower()
    found = False
    query = content + " " + author
    for key in term_dict.keys():
      if any(term in query for term in term_dict[key]):
        df_dict[key].loc[len(df_dict[key])] = row
        found = True
        break
    if not found:
      df_dict["other"].loc[len(df_dict["other"])] = row


In [130]:
# prompt: make function to save dataframe into csv with the name same as the variable name
def save_df_to_csv(df_dict):
  for k, v in df_dict.items():
    v.to_csv(f"{k}.csv", index=False)


In [186]:
classify_df(df, df_dict)

In [187]:
len(df_dict["other"])

50

In [188]:
len(df_dict["math"])+len(df_dict["idol"])+len(df_dict["tech"])+len(df_dict["other"])

349

In [189]:
# Call the function for each DataFrame
save_df_to_csv(df_dict)